# Compte Rendu du TP2 de Système de Recommandations

### Auteurs : Boubacar TRAORE & Zakarya JARRAYA

In [278]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [273]:
# Changement de la police utilisée et de sa taille
from IPython.core.display import HTML, display
from tp_tools import change_font
display(HTML(change_font()))

In [274]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

## 1. Comparaison des 3 algorithmes

In [291]:
# Packages
from exp import games, cumulative_regret, cumulative_reward, logarithmic_indices
from random import shuffle
from player import Oracle, EpsilonNGreedy, ThompsonSamplingBernoulli, UCB1, ExploreThenCommit
from arm import MyBeta, Bernoulli
import numpy as np
import matplotlib.pyplot as plt
# Librairies de visualisation des résultats
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)

### 1.1. Création de l'environnement

Nous récupérons les informations des 10 bras sur Moodle. Nous fixons le nombre d'essais total de chaque jeu à $T = 10K$, un nombre assez important pour constater l'évolution du regret cumulé lorsque T devient très important (**n_iter dans le code ci-dessous**). Pour l'instant on garde le nombre de jeu à effectuer par chaque algorithme à **n_games = 10** : c'est la courbe moyenne sur l'ensemble des 10 jeux qui sera représenté pour chaque algorithme.

**NB** : Les bras sont tous simulés selon une loi **beta** ($\beta$) dans cette cette section.

In [3]:
environment = [MyBeta(mean=0.44), MyBeta(mean=0.52), MyBeta(mean=0.87), MyBeta(mean=0.69), MyBeta(mean=0.32), 
               MyBeta(mean=0.01), MyBeta(mean=0.24), MyBeta(mean=0.65), MyBeta(mean=0.49), MyBeta(mean=0.68)]

# Odonner aléatoirement les bras
shuffle(environment)

#Paramètres
n_iter = 1000
n_games = 10

### 1.2. Implémentation et représentation des algorithmes

In [138]:
#Oracle
log_oracle = games(Oracle(np.argmax([a.mean() for a in environment])), environment, n_iter, n_games)


# Epsilon Greedy
c_EG = 1
log_EG1    = games(EpsilonNGreedy(nb_arms=len(environment), c=c_EG), environment, n_iter, n_games)

## Thompson Sampling
log_TS = games(ThompsonSamplingBernoulli(nb_arms=len(environment), prior_s=0.25, prior_f=0.75), environment, n_iter, n_games)

#UCB 1
c_ucb = 2
log_UCB1 = games(UCB1(nb_arms=len(environment), c=c_ucb), environment, n_iter, n_games)

Nous définissons une fonction globale qui permet de représenter.

In [139]:
def plot_cumulative_regret(logs, labels, n_iter=1000):
    """Représentation du regret cumulé des algorithmes données en input.

    Arguments:
        logs {list} -- Liste des logs des algorithmes de bandits manchots.
        labels {list} -- Liste des labels pour la représentation graphique.

    Keyword Arguments:
        n_iter {int} -- Nombre d'essais par jeu (default: {1000})
        sub_plots {tuple}  -- Nombre de sous-graphes à faire. Nombre de fois à répéter tous les logs (default: {(1,1)})

    Returns:
        plot -- plotly object for graph
    """
    
    inds = logarithmic_indices(n_iter, 100) #Afficher seulement 100 points au toytal

    fig = go.Figure() #Figure plotly
    for i in range(len(logs)):
        plot = go.Scatter(x=inds + 1, y=cumulative_regret(logs[i])[inds], mode = 'lines', name = labels[i])
        fig.add_trace(plot)
    #Mise en forme    
    fig['layout'] = go.Layout(title="Evolution du Regret cumulé selon le nombre d'itérations",
                              xaxis=go.layout.XAxis(title="Nombre d'essais"),
                              yaxis=go.layout.YAxis(title='Regret Cumulé'))

    return iplot(fig)

Exécutons la fonction avec les informations requises.

In [140]:
logs   = [log_oracle, log_EG1, log_UCB1, log_TS]
labels = ["Oracle", "EG, c=1", "UCB1, c=2", "TS"]
plot_cumulative_regret(logs=logs, labels=labels)

A première vue, l'algorithme de Thompson Sampling semble nettement meilleur que ceux de l'UCB1 (avec c=2) et d'Epsilon-Greedy (avec c=1).

### 1.3. Choix de l'hyper-paramètre "c" dans Epsilon Greedy.

L'algorithme choisit de faire de l'exploration souvent selon une probabilité faible fixé généralement à $\epsilon = 0.05$. Dans notre cas spécifique, nous écrivons $\epsilon$ comme une fonction décroissante du nombre d'essais afin d'exploiter au mieux le bras ayant la meilleure espérance de gain lorsque le jeu avance : $\epsilon \approx \frac{c}{t}$ où $t \in [|1,nb\_trials|]$. Trouver une bonne valeur de $c$ est donc important pour la performance de l'algorithme. C'est un hyper-paramètre dont la valeur optimale dépend de l'environnement dans lequel nous jouns (l'ensemble des bras utilisés). Ici, nous proposons une méthode de sélection de cette valeur.

Dans un premier temps, nous n'allons pas représenter l'Oracle et les autres algorithmes (TS et UCB1), nous nous focalisons juste sur l'epsilon-greedy en faisant varier le paramètre $c$.

#### 1.3.1. Choix aléatoire selon une valeur maximale de "c=50"

$\epsilon \approx \frac{c}{t}$ signifie que l'algorithme ferra toujours de l'exploration avant que le nombre d'essais $t$ n'atteigne la valeur de $c$ (on suppose, au moment de l'implémentation de l'algorithme, que si $\epsilon >1$ l'algorithme ne fait qu'explorer, et donc choisit aléatoirement parmi les bras). Etant donné que nous qu'un jeu se fait jusqu'à 1000 essais, une valeur de $c=50$ semble tout fait grand pour explorer. L'algorithme se décrit donc comme suit :
* Si $t\leq50$, on fait de l'exploration (choix aléatoire entre les 10 bras disponibles)
* Si $t>50$, on fait de l'exploitation avec une probilité de $p = 1 - \frac{50}{t}$.

Nous allons choisir aléatoirement 10 valeurs de $c$ selon une loi uniforme [0,50] et représenter l'évolution du regret cumulé pour chacun d'entre eux.

**Remarque**: Nous allons beaucoup augmenter le nombre de jeu pour chaque valeur de $c$ afin que les courbes soient les plus stables que possible (loi des grands nombres). On prendra **nb_games = 100**. Les courbes tracées pour chaque valeur de $c$ ne seront autre que les moyennes empiriques des 100 courbes de jeu pour chacun d'entre eux. Augmenter le nombre de jeux assure une certaine fiabilité des résultats mais demande aussi un temps de calcul non négligeable (nous avons voulu en prendre plus mais ça prenait beaucoup de temps, vaudrait donc mieux se limiter à 100).

In [105]:
n_games = 100 #Augmentation du nombre de jeux
np.random.seed(26) #Fixer la graine générateur pour avoir un résultat réplicable
c_range = sorted(np.random.randint(low=1, high=50, size=10))
logs_EG = []
labels  = []
for c in c_range:
    logs_EG.append(games(EpsilonNGreedy(nb_arms=len(environment), c=c), environment, n_iter, n_games))
    labels.append('EG, c = %d'%c)
plot_cumulative_regret(logs=logs_EG, labels=labels)

Nous remarquons que pour $c=2$, il n'y a vraiment pas assez d'exploration, ceci est interprété par une courbe presque linéaire. Cette valeur de $c$ est donc très faible. D'un autre côté, pour les valeurs de $c\ge27$, il y a trop d'exploration et les courbes de regret cumulées décollent très vite et on du mal à se stabiliser lorsque le nombre d'essais grandit. Avec un nombre d'essais $T = 1000$, ce sont les valeurs $c=5$ et $c=7$ qui enregistrent les meilleures performances pour cet algorithme (même si $c=5$ gagne légèrement).

Nous allons maintenant appliquer deux grandes opérations :
* Zoomer dans la zone des entiers compris entre 3 et 25 (valeurs assez raisonables d'après le graphique précédent) comme valeurs possibles de $c$
* Doubler le nombre d'essais à $T=2000$ pour mieux distinguer la performance à long terme de l'algorirhme pour ces différentes valeurs.

#### 1.3.2. Choix avec une recherche quasi-exhaustive des valeurs entières entre [3,25]

Dans ce problème, nous nous limitons aux nombres entiers comme valeurs possibles de $c$ afin de ne pas complexifier le problème, sinon le raisonnement peut bien s'étendre aux nombres réels. Nous allons donc parcourir toutes les valeurs de $c$ comprises entre 3 et 25 avec un pas de 2.

In [111]:
n_games = 100 #Augmentation du nombre de jeux
n_iter  = 2000 # Doubler le nombre d'itérations
c_range = range(3,26,2)
logs_EG = []
labels  = []
for c in c_range:
    logs_EG.append(games(EpsilonNGreedy(nb_arms=len(environment), c=c), environment, n_iter, n_games))
    labels.append('EG, c = %d'%c)
plot_cumulative_regret(logs=logs_EG, labels=labels, n_iter=n_iter)

Visiblement, les valeurs de $c\leq5$ ne sont pas bonnes (vu leurs pentes). $c=11$ semble se distinguer des autres et possède la meilleure performance (nous augmeterons le nombre de jeu. Les autres valeurs ne semblent pas mal non plus. Pour finaliser cette recherche, nous allons encore raffiner l'intervalle en nous foclisant uniquement sur les valeurs de $c$ qui ont cumulé moins de regret et dont les pentes sont quasi-nulles au bout de 2000 essais.

#### 1.3.3. Dernier raffinage sur [10,20]

Une fois de plus nous allons zommer sur [10,20] et augmenter le nombre d'itérations. Pour encore être plus précis, nous fixons le nombre de jeux 200 (2 fois plus que la dernière fois).

In [115]:
n_games = 200 #Doubler du nombre de jeux par rapport à la dernière fois
n_iter  = 3000 # Augmenter le nombre d'itérations de 1000 encore.
c_range = range(10,21)
logs_EG = []
labels  = []
for c in c_range:
    logs_EG.append(games(EpsilonNGreedy(nb_arms=len(environment), c=c), environment, n_iter, n_games))
    labels.append('EG, c = %d'%c)
plot_cumulative_regret(logs=logs_EG, labels=labels, n_iter=n_iter)

Comme le résultat varie souvent lorsque nous réexécutons la même cellule, nous avons stockés les 5 meilleurs valeurs de $c$ pour 5 exécutions différentes. Les valeurs sont recenssés ci-dessous:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-dvpl{border-color:inherit;text-align:right;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-0pky"></th>
    <th class="tg-c3ow" colspan="5">Rang</th>
  </tr>
  <tr>
    <td class="tg-dvpl">i-ème exécution</td>
    <td class="tg-0pky">1er</td>
    <td class="tg-0pky">2e</td>
    <td class="tg-0pky">3e</td>
    <td class="tg-0pky">4e</td>
    <td class="tg-0pky">5e</td>
  </tr>
  <tr>
    <td class="tg-dvpl">1</td>
    <td class="tg-0pky">16</td>
    <td class="tg-0pky">14</td>
    <td class="tg-0pky">13</td>
    <td class="tg-0pky">12</td>
    <td class="tg-0pky">17</td>
  </tr>
  <tr>
    <td class="tg-dvpl">2</td>
    <td class="tg-0pky">16</td>
    <td class="tg-0pky">18</td>
    <td class="tg-0pky">17</td>
    <td class="tg-0pky">14</td>
    <td class="tg-0pky">13</td>
  </tr>
  <tr>
    <td class="tg-dvpl">3</td>
    <td class="tg-0pky">13</td>
    <td class="tg-0pky">17</td>
    <td class="tg-0pky">18</td>
    <td class="tg-0pky">14</td>
    <td class="tg-0pky">15</td>
  </tr>
  <tr>
    <td class="tg-dvpl">4</td>
    <td class="tg-0pky">13</td>
    <td class="tg-0pky">14</td>
    <td class="tg-0pky">10</td>
    <td class="tg-0pky">15</td>
    <td class="tg-0pky">17</td>
  </tr>
  <tr>
    <td class="tg-dvpl">5</td>
    <td class="tg-0pky">12</td>
    <td class="tg-0pky">14</td>
    <td class="tg-0pky">15</td>
    <td class="tg-0pky">8</td>
    <td class="tg-0pky">13</td>
  </tr>
</table>

Nous voyons bien que ce sont les chiffres 13 et 14 qui se répètent beaucoup. Nous pouvons choisir 13 comme valeur à retenir. On retient qu'à long terme le 13 est plus performant que le 11 retenu dans la section précédente.

### 1.4. Nouvelle comparaison des algorithmes avec la valeur optimale de c trouvée précédemment

In [137]:
n_games = 200
n_iter  = 3000

#Oracle
log_oracle = games(Oracle(np.argmax([a.mean() for a in environment])), environment, n_iter, n_games)

# Epsilon Greedy
c_EG = 13
log_EG = games(EpsilonNGreedy(nb_arms=len(environment), c=c_EG), environment, n_iter, n_games)

## Thompson Sampling
log_TS = games(ThompsonSamplingBernoulli(nb_arms=len(environment), prior_s=0.25, prior_f=0.75), environment, n_iter, n_games)

#UCB1
c_ucb = 1
log_UCB1 = games(UCB1(nb_arms=len(environment), c=c_ucb), environment, n_iter, n_games)

logs   = [log_oracle, log_EG, log_UCB1, log_TS]
labels = ["Oracle", "EG, c=11", "UCB1, c=1", "TS"]
plot_cumulative_regret(logs=logs, labels=labels, n_iter=n_iter)

Nous pouvons remarquer que l'algorithme d'Epsilon Greedy a drastiquement diminué sa différence de regret cumulé face à l'algorithme de Thompson Sampling qui reste toujours meilleur. De la même manière, il possible de bien tuner l'hyper-paramètre $c$ de l'algorithme UCB1 pour atteindre de meilleurs performances.

**Comparaisons générales**

* Le Thompson Sampling reste globalement le meilleur algorithme. Elle fait assez d'exploration au départ et sa phase d'exploitation est excellente.
* L'UCB1 est aussi réputé pour avoir de bons résultats malgré qu'il n'a quasiment aucum paramètre à tuner (on peut généralement fixer la constante d'UCB1 à $c\_ucb=2$, quoique dans notre cas elle ne donne pas d'assez bons résultats).
* Epsilon Greedy, avec de bons paramètres, a quasiment la même performance que Thompson Sampling. 

## 2. Bonus : Bernouilli arms

L'implémentation est effectuée dans la classe correspondante. Nous redéfinissons npotre environnement avec toujours 10 bras, mais cette fois-ci les résultats des bras sont tirés selon une loi bernoulli.

In [289]:
environment = [Bernoulli(p=0.2), Bernoulli(p=0.75), Bernoulli(p=0.9), Bernoulli(p=0.3), Bernoulli(p=0.7), 
               Bernoulli(p=0.1), Bernoulli(p=0.8), Bernoulli(p=0.4), Bernoulli(p=0.6), Bernoulli(p=0.5)]

In [299]:
n_games = 50
n_iter  = 1000

#Oracle
log_oracle = games(Oracle(np.argmax([a.mean() for a in environment])), environment, n_iter, n_games)

# Epsilon Greedy
c_EG = 13
log_EG = games(EpsilonNGreedy(nb_arms=len(environment), c=c_EG), environment, n_iter, n_games)

## Thompson Sampling
log_TS = games(ThompsonSamplingBernoulli(nb_arms=len(environment), prior_s=0.25, prior_f=0.75), environment, n_iter, n_games)

##ETC
log_ETC = games(ExploreThenCommit(nb_arms=len(environment), n0=300), environment, n_iter, n_games)

#UCB1
c_ucb = 1
log_UCB1 = games(UCB1(nb_arms=len(environment), c=c_ucb), environment, n_iter, n_games)

logs   = [log_oracle, log_EG, log_UCB1, log_TS, log_ETC]
labels = ["Oracle", "EG, c=11", "UCB1, c=1", "TS", "ETC, n0 = 300"]
plot_cumulative_regret(logs=logs, labels=labels, n_iter=n_iter)

Avec cette configuration, nous remarquons que tous les algorithmes sont plus performants que l'ETC. Ceci est dû au fait qu'on lui donne trop de temps d'exploration, il passe donc $\frac{9}{10}$ de son temps avant $n_0$ à chosir un mauvais bras.